In [1]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models,Input, Model,applications
from PIL import Image
import os
from tensorflow.keras.losses import MeanSquaredError, MeanSquaredLogarithmicError,MeanAbsolutePercentageError,CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

import glob
import gc

from google.colab import drive
from google.colab import files

In [2]:
drive.mount("/content/drive", force_remount=True)
%cd drive/MyDrive/Colab Notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [3]:
# im = Image.open('a_image.tif')
# im.show()
# imarray = numpy.array(im)
#File_data = np.loadtxt("example1.txt", dtype=np.float32)    
# BatchNormalization
#padding="same"

# **load Data**

In [5]:
pathX = "./data2/TrainImages/"
pathY = "./data2/TrainGroundTruth/"

# imgsY = os.listdir(pathY)
x_train = []
y_train = []

imgsX = glob.glob(pathX+'*')
count = len(imgsX)
del imgsX
gc.collect()
k=0
for i in range(count):
    # print(i)
   
    f = pathX+str(i+1)+".tif"
    ff = glob.glob(f)
    if len(ff) == 0: continue


    
    y = np.loadtxt(pathY+str(i+1)+".txt", delimiter=',', dtype=np.int16)
    if y.ndim > 1:
        k+=1
        # y = y[0,:]
        continue
        
    # y = np.concatenate([np.ones((1)),y])
    y = np.reshape(y,(1,4)).astype(np.float32)
    y_train.append(y/1000)


    im = Image.open(f)
    img = im.resize((350,350))#resize
    img = np.array(img)
    img = img.astype(np.float32)/255
    x_train.append(img)

x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = np.squeeze(y_train)

In [ ]:
pathX = "./data2/TestImages/"
pathY = "./data2/TestGroundTruth/"

# imgsY = os.listdir(pathY)
x_test = []
y_test = []

imgsX = glob.glob(pathX+'*')
count = len(imgsX)+680
del imgsX
gc.collect()



k=0
for i in range(679,count):
    
    f = pathX+str(i+1)+".tif"
    ff = glob.glob(f)
    if len(ff) == 0: continue
    

    
    y = np.loadtxt(pathY+str(i+1)+".txt", delimiter=',', dtype=np.float32)
    if y.ndim > 1:
        k+=1
        # y = y[0,:]
        continue
        
    y = np.reshape(y,(1,4))
    y_test.append(y/1000)#/5
    
    im = Image.open(f)
    im = im.resize((400,400))#resize
    img = np.array(im)/255
    x_test.append(img)


x_test = np.array(x_test)
y_test = np.array(y_test)
y_test = np.squeeze(y_test)

In [6]:
def bb_iou(boxA, boxB):

	xA = np.maximum(boxA[:,0], boxB[:,0])
	yA = np.maximum(boxA[:,1], boxB[:,1])
	xB = np.minimum(boxA[:,2], boxB[:,2])
	yB = np.minimum(boxA[:,3], boxB[:,3])
	# compute the area of intersection rectangle
	interArea = np.maximum(0, xB - xA + 1) * np.maximum(0, yB - yA + 1)

	boxAArea = (boxA[:,2] - boxA[:,0] + 1) * (boxA[:,3] - boxA[:,1] + 1)
	boxBArea = (boxB[:,2] - boxB[:,0] + 1) * (boxB[:,3] - boxB[:,1] + 1)

	iou = interArea / (boxAArea + boxBArea - interArea)

	return iou.mean()#np.sum(iou)/641

In [7]:
def bb_iou_(boxA, boxB):

	xA = max(boxA[:,0], boxB[:,0])
	yA = np.maximum(boxA[:,1], boxB[:,1])
	xB = np.minimum(boxA[:,2], boxB[:,2])
	yB = np.minimum(boxA[:,3], boxB[:,3])
	# compute the area of intersection rectangle
	interArea = np.maximum(0, xB - xA + 1) * np.maximum(0, yB - yA + 1)

	boxAArea = (boxA[:,2] - boxA[:,0] + 1) * (boxA[:,3] - boxA[:,1] + 1)
	boxBArea = (boxB[:,2] - boxB[:,0] + 1) * (boxB[:,3] - boxB[:,1] + 1)

	iou = interArea / (boxAArea + boxBArea - interArea)

	return iou

# **model1====================================**

In [22]:
def model():
  input_ = Input(shape=(350,350,3))
  # x = layers.BatchNormalization()(input_)

  x = layers.Conv2D(filters=16,kernel_size=7,padding="same",activation='relu')(input_)
  # x = layers.BatchNormalization()(x)


  x = layers.Conv2D(filters=32,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.Conv2D(filters=32,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.Conv2D(filters=32,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.MaxPooling2D(pool_size= 2, strides = 2)(x)
  # x = layers.BatchNormalization()(x)

  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.MaxPooling2D(pool_size= 4, strides = 4)(x)
  # x = layers.BatchNormalization()(x)

  x = layers.Conv2D(filters=128,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.Conv2D(filters=128,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.Conv2D(filters=128,kernel_size=(3,3),padding="valid",activation='relu')(x)
  x = layers.AveragePooling2D(pool_size= 5, strides = 3)(x)
  # x = layers.BatchNormalization()(x)

  # x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same",activation='relu')(x)
  # x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same",activation='relu')(x)
  # x = layers.MaxPooling2D(pool_size= 3, strides = 2)(x)

  # x = layers.Flatten()(x)
  x =  layers.Flatten()(x)


######## bounding box
  # x1 = layers.Dense(256,activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5)) (x)
  # x1 = layers.BatchNormalization()(x1)
  # x1 = layers.Dropout(0.4)(x1)
  x1 = layers.Dense(256,activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5)) (x)
  
  x1 = layers.Dense(128,activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5)) (x1)


  x1 = layers.Dense(128,activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5)) (x1)

  x1 = layers.Dense(4,activation='sigmoid') (x1)

######## confidence
  # x2 = layers.Dense(64,activation='relu') (x)
  # x2 = layers.Dropout(0.4)(x2)
  # x2 = layers.Dense(32,activation='relu') (x2)
  # x2 = layers.Dropout(0.2)(x2)
  # x2 = layers.Dense(4,activation='linear') (x2)
  # output_layer = layers.concatenate([x_2,x1], axis = -1)

  model = Model(input_, x1)

  return model

In [23]:
cnn = model()
cnn.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 350, 350, 3)]     0         
                                                                 
 conv2d_30 (Conv2D)          (None, 350, 350, 16)      2368      
                                                                 
 conv2d_31 (Conv2D)          (None, 348, 348, 32)      4640      
                                                                 
 conv2d_32 (Conv2D)          (None, 346, 346, 32)      9248      
                                                                 
 conv2d_33 (Conv2D)          (None, 344, 344, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 172, 172, 32)     0         
 2D)                                                             
                                                           

In [24]:
cnn.compile(
optimizer=Adam(learning_rate=1e-4,),
loss=MeanAbsolutePercentageError(),
metrics=[tf.keras.metrics.MeanAbsoluteError(),],)

In [25]:
cnn.fit(x_train,y_train, batch_size=64, epochs=400)

Epoch 1/400
9/9 [==============================] - 33s 2s/step - loss: 39.9210 - mean_absolute_error: 0.1638
Epoch 2/400
9/9 [==============================] - 9s 1s/step - loss: 36.3443 - mean_absolute_error: 0.1432
Epoch 3/400
9/9 [==============================] - 9s 1s/step - loss: 35.5870 - mean_absolute_error: 0.1351
Epoch 4/400
9/9 [==============================] - 9s 994ms/step - loss: 35.6065 - mean_absolute_error: 0.1374
Epoch 5/400
9/9 [==============================] - 9s 991ms/step - loss: 35.5714 - mean_absolute_error: 0.1413
Epoch 6/400
9/9 [==============================] - 9s 992ms/step - loss: 35.0889 - mean_absolute_error: 0.1332
Epoch 7/400
9/9 [==============================] - 9s 993ms/step - loss: 34.5259 - mean_absolute_error: 0.1282
Epoch 8/400
9/9 [==============================] - 9s 1s/step - loss: 32.2036 - mean_absolute_error: 0.1172
Epoch 9/400
9/9 [==============================] - 9s 1s/step - loss: 28.1692 - mean_absolute_error: 0.1096
Epoch 10/400
9/

KeyboardInterrupt: ignored

In [26]:
p = cnn.predict(x_train)
bb_iou(p*1000,y_train*1000)

18/18 [==============================] - 58s 161ms/step


0.4855689

In [ ]:
p = cnn.predict(x_test)
bb_iou(p*1000,y_test*1000).mean()

NameError: ignored

# **model2=============================================**


In [ ]:
# IMG_SHAPE = (200, 200, 3)
# InceptionV3 = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

In [ ]:
IMG_SHAPE = (400, 400, 3)
MobileNetV2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
MobileNetV2.summary()

In [ ]:
def model2():
  MobileNetV2.trainable = False
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  # x = applications.MobileNet_V2.preprocess_input(inputs)
  x = MobileNetV2(inputs)
  # x = InceptionV3(inputs)
  x = layers.Conv2D(filters=512,kernel_size=7,padding="same",activation='relu')(x)

  x = layers.Conv2D(filters=256,kernel_size=3,padding="same",activation='relu')(x)
  x = layers.Conv2D(filters=256,kernel_size=3,padding="same",activation='relu')(x)
  x = layers.Conv2D(filters=256,kernel_size=3,padding="same",activation='relu')(x)
  
  x = layers.Flatten()(x)
  # x = layers.BatchNormalization()(x)
  
  x1 = layers.Dense(512,activation='relu',kernel_regularizer=regularizers.L1L2(l1=0, l2=1e-5),) (x)

  x1 = layers.Dense(256,activation='relu',kernel_regularizer=regularizers.L1L2(l1=0, l2=1e-5),) (x1)

  x1 = layers.Dense(128,activation='relu',kernel_regularizer=regularizers.L1L2(l1=0, l2=1e-5),) (x1)

  # x1 = layers.Dense(128,activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-3),) (x1)

  # x1 = layers.Dropout(0.01)(x1)
  x1 = layers.Dense(4,activation='sigmoid') (x1)

  return  Model(inputs,x1)

In [ ]:
cnn = model2()
cnn.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 400, 400, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 13, 13, 1280)     2257984   
 ional)                                                          
                                                                 
 conv2d_173 (Conv2D)         (None, 13, 13, 512)       32113152  
                                                                 
 conv2d_174 (Conv2D)         (None, 13, 13, 256)       1179904   
                                                                 
 conv2d_175 (Conv2D)         (None, 13, 13, 256)       590080    
                                                                 
 conv2d_176 (Conv2D)         (None, 13, 13, 256)       590080    
                                                          

In [ ]:
cnn.compile(
optimizer=Adam(learning_rate=1e-4,),
loss=MeanAbsolutePercentageError(),
metrics=[tf.keras.metrics.MeanAbsoluteError()],)

In [ ]:
cnn.fit(x_train,y_train,batch_size= 32,epochs=100)

Epoch 1/100
18/18 [==============================] - 36s 304ms/step - loss: 37.2204 - mean_absolute_error: 0.1586
Epoch 2/100
18/18 [==============================] - 5s 293ms/step - loss: 25.0779 - mean_absolute_error: 0.0994
Epoch 3/100
18/18 [==============================] - 5s 295ms/step - loss: 15.4351 - mean_absolute_error: 0.0683
Epoch 4/100
18/18 [==============================] - 5s 294ms/step - loss: 10.5695 - mean_absolute_error: 0.0520
Epoch 5/100
18/18 [==============================] - 5s 295ms/step - loss: 7.8816 - mean_absolute_error: 0.0404
Epoch 6/100
18/18 [==============================] - 5s 296ms/step - loss: 6.2224 - mean_absolute_error: 0.0322
Epoch 7/100
18/18 [==============================] - 5s 296ms/step - loss: 5.5893 - mean_absolute_error: 0.0287
Epoch 8/100
18/18 [==============================] - 5s 297ms/step - loss: 5.3326 - mean_absolute_error: 0.0278
Epoch 9/100
18/18 [==============================] - 5s 298ms/step - loss: 4.9160 - mean_absolute_e

KeyboardInterrupt: ignored

In [ ]:
cnn.save('ODmodels/v3')

In [ ]:
p = cnn.predict(x_train)

bb_iou(p*1000,y_train*1000),bb_iou_(p,y_train)

18/18 [==============================] - 4s 181ms/step


(0.62663, array([0.9512072 , 0.928586  , 0.94869393, 0.9773692 , 0.97796786,
        0.95070475, 0.95463043, 0.9541173 , 0.959165  , 0.95665634,
        0.955168  , 0.9905154 , 0.94523484, 0.9478421 , 0.9831998 ,
        0.9669066 , 0.870956  , 0.9663938 , 0.9511556 , 0.9576847 ,
        0.96522087, 0.98000234, 0.97264135, 0.962946  , 0.95509946,
        0.95788103, 0.97978216, 0.9653176 , 0.9243551 , 0.953944  ,
        0.96735495, 0.95995593, 0.9498798 , 0.96209836, 0.96857166,
        0.97018826, 0.96347606, 0.96854895, 0.964982  , 0.96384907,
        0.9600355 , 0.9699953 , 0.9501814 , 0.93105495, 0.95071876,
        0.97188276, 0.96934116, 0.9667816 , 0.9465085 , 0.95056206,
        0.9613905 , 0.9863463 , 0.95748544, 0.96360594, 0.97504866,
        0.9756247 , 0.96280617, 0.947444  , 0.9764372 , 0.94516027,
        0.9632501 , 0.9519916 , 0.96051306, 0.9604565 , 0.96863294,
        0.98274577, 0.95157695, 0.98453784, 0.98142576, 0.9542964 ,
        0.9759877 , 0.9753223 , 0.96902

In [ ]:
p = cnn.predict(x_test)

bb_iou(p*1000,y_test*1000),bb_iou_(p,y_test)

4/4 [==============================] - 1s 209ms/step


(0.3318147, array([0.90319777, 0.8735412 , 0.9226432 , 0.95421845, 0.8383285 ,
        0.94489866, 0.94787955, 0.97467476, 0.6670854 , 0.8879846 ,
        0.9341702 , 0.8992302 , 0.8888175 , 0.97858256, 0.97065836,
        0.9065342 , 0.8387836 , 0.7386612 , 0.90941423, 0.9078957 ,
        0.90729356, 0.9499653 , 0.85897   , 0.94892454, 0.94134027,
        0.9293499 , 0.89404434, 0.93676263, 0.5609236 , 0.7899321 ,
        0.8628964 , 0.9475987 , 0.93794537, 0.8755313 , 0.96027184,
        0.89916486, 0.89258224, 0.8786591 , 0.8227517 , 0.9150634 ,
        0.948305  , 0.95862746, 0.75703764, 0.99471927, 0.9262381 ,
        0.69573146, 0.887153  , 0.9025374 , 0.90651846, 0.9442332 ,
        0.8544045 , 0.96066105, 0.9635926 , 0.9089195 , 0.82602423,
        0.8903819 , 0.93259877, 0.88433194, 0.9508079 , 0.95474195,
        0.7667056 , 0.68229365, 0.8814998 , 0.69691616, 0.91279966,
        0.9058039 , 0.94719774, 0.92255265, 0.91468793, 0.89718735,
        0.8909035 , 0.8426005 , 0.940

In [ ]:
x_train.shape